Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
import requests
import pandas as pd

# citybikes results
citybikes_df = pd.read_csv('citybikes_data.csv')

# Foursquare results
foursquare_df = pd.read_csv('fsq_data.csv')

# Yelp results
yelp_df = pd.read_csv('yelp_datas.csv')

# Merge dataframes
merged_df = pd.concat([citybikes_df, foursquare_df, yelp_df], axis=0, ignore_index=True)
print(merged_df)

In [120]:
import pandas as pd

# Load your CSV file into a DataFrame
data = pd.read_csv('fsq_data.csv')

# Group the data by latitude and longitude and count the number of parks for each group
data['fsq_table'] = data.groupby(['lat', 'long'])['park_name'].transform('count')

# Save the updated DataFrame to a new CSV file
data.to_csv('fsq_park.csv', index=False)


Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [4]:
import folium

# map centered around London
map_london = folium.Map(location=[51.5074, -0.1278], zoom_start=12)

# Adding markers for each data point in citybikes dataframe
for index, row in citybikes_df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],  # Replace with the appropriate column from your dataframe
        icon=folium.Icon(color='blue')
    ).add_to(map_london)

# Display the map
map_london

In [ ]:
plt.figure(figsize=(8, 6))  # Adjust the figure size as per your preference
plt.scatter(merged_df['Longitude'], merged_df['Latitude'])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Distribution of Stations in London')
plt.show()

In [ ]:
We can see that there are more bikes in central London.

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [117]:
import sqlite3
from sqlite3 import Error
# using sqlalchemy
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection
connection = create_connection("sm_app.sqlite")

Connection to SQLite DB successful


In [101]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [102]:
import pandas as pd
from sqlalchemy import create_engine
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

connection = create_connection("new_data.db")
citybikes_df = pd.DataFrame(station_data)
foursquare_df = pd.DataFrame(fsq_data)
yelp_df = pd.DataFrame(yelp_datas)

citybikes_df.to_sql('citybikes', con=engine, if_exists='replace', index=False)
foursquare_df.to_sql('foursquare', con=engine, if_exists='replace', index=False)
yelp_df.to_sql('yelp', con=engine, if_exists='replace', index=False)

print("Data added to the SQLite database.")

Connection to SQLite DB successful
Data added to the SQLite database.


Look at the data before and after the join to validate your data.

In [103]:
import pandas as pd

#convert dataframe to csv
citybikes_df.to_csv('citybikes.csv', index=False)
yelp_df.to_csv('yelp.csv', index=False)
foursquare_df.to_csv('foursquare.csv', index=False)


In [112]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.sql import select

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

connection = create_connection("new_data.db")

#  the base class
Base = sqlalchemy.orm.declarative_base()

#table classes representing the dataframes
class CityBikes(Base):
    __tablename__ = 'citybikes'
    id = Column(Integer, primary_key=True)
    station_id = Column(Integer)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    bikes = Column(Integer)

class Foursquare(Base):
    __tablename__ = 'foursquare'
    id = Column(Integer, primary_key=True)
    poi_name = Column(String)
    rating = Column(Float)
    fsq_id = Column(String)
    longitude = Column(Float)
    latitude = Column(Float)

class Yelp(Base):
    __tablename__ = 'yelp'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    rating = Column(Float)
    review_count = Column(Integer)
    phone = Column(String)
    location = Column(String)
    city = Column(String)
    zip_code = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    category = Column(String)

# the database engine
engine = create_engine('sqlite:///data/new_data.db', echo=True)

#table schemas
Base.metadata.create_all(engine)

# session object
Session = sessionmaker(bind=engine)
session = Session()

# Perform the join query using SQLAlchemy
join_query = session.query(CityBikes, Foursquare, Yelp).filter(
    CityBikes.longitude == Foursquare.longitude,
    CityBikes.latitude == Foursquare.latitude,
    CityBikes.longitude == Yelp.longitude,
    CityBikes.latitude == Yelp.latitude
).all()

# Store the results in a list
joined_data = []
for citybikes, foursquare, yelp in join_query:
    joined_data.append({
        "Station ID": citybikes.station_id,
        "Name": citybikes.name,
        "Latitude": citybikes.latitude,
        "Longitude": citybikes.longitude,
        "Bikes": citybikes.bikes,
        "POI Name": foursquare.poi_name,
        "Rating": foursquare.rating,
        "FSQ_ID": foursquare.fsq_id,
        "Yelp Name": yelp.name,
        "Yelp Rating": yelp.rating,
        "Review Count": yelp.review_count,
        "Phone": yelp.phone,
        "Location": yelp.location,
        "City": yelp.city,
        "Zip Code": yelp.zip_code,
        "Category": yelp.category
    })

# Convert the joined data to a pandas DataFrame
joined_df = pd.DataFrame(joined_data)

# Validate the joined DataFrame
print(joined_df.head())

#missing values
missing_values = pd.isnull(joined_df).sum()